# Detecting change in Australian forestry <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Products used:** 
[s2_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), 
[s2b_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule)

## Background
Effective management of Australia's forests is critical for balancing environmental protection and sustainable growth of the industry.
Methods for detecting meaningful and significant change in forests are important for those who manage and monitor large areas of forest.

On-the-ground monitoring can be expensive and time-consuming, especially when forests are in difficult-to-navigate terrain.
Aerial photography and LiDAR can provide detailed information about forests, but are often extremely expensive to acquire, even over small areas.

### Sentinel-2 use case
Satellite imagery from the [EU Copernicus Sentinel-2 mission](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) is freely available and has a revisit time over Australia of ~5 days.
Its 10 metre resolution makes it perfect for monitoring fine changes over very large areas of land.
The archive of Sentinel-2 data stretches back to 2015, meaning that there is a good amount of data for change detection, allowing us to average out or focus on seasonal changes.

## Description
In this example, we measure the presence of vegetation from Sentinel-2 imagery and apply a hypothesis test to identify areas of significant change (along with the direction of the change).
The worked example takes users through the code required to do the following:

1. Load cloud-free Sentinel-2 images for an area of interest
2. Compute an index to highlight presence of vegetation (NDVI)
3. Apply a statistical hypothesis test to find areas of significant change
4. Visualise the statistically significant areas.

***

## Getting started

**To run this analysis**, run all the cells in the notebook, starting with the "Load packages" cell.

**After finishing the analysis**, return to the "Analysis parameters" cell, modify some values (e.g. choose a different location or time period to analyse) and re-run the analysis.
There are additional instructions on modifying the notebook at the end.

### Load packages
Load key Python packages and any supporting functions for the analysis.

In [1]:
%matplotlib inline

import sys
import datacube
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from datacube.helpers import write_geotiff
from datacube.utils.geometry import CRS

sys.path.append("../scripts")
from dea_datahandling import load_ard
from dea_bandindices import calculate_indices
from dea_plotting import display_map
from dea_plotting import rgb

### Connect to the datacube
Activate the datacube database, which provides functionality for loading and displaying stored Earth observation data.

In [2]:
dc = datacube.Datacube(app="Change_detection")

### Analysis parameters

The following cell sets the parameters, which define the area of interest and the length of time to conduct the analysis over.
There is also a parameter to define how the data is split in time; the split yields two non-overlapping samples, which is a requirement of the hypothesis test we want to run (more detail below).
The parameters are:

* `latitude`: The latitude range to analyse (e.g. `(-35.271, -35.331)`).
For reasonable loading times, make sure the range spans less than ~0.1 degrees.
* `longitude`: The longitude range to analyse (e.g. `(149.256, 149.356)`).
For reasonable loading times, make sure the range spans less than ~0.1 degrees.
* `time`: The date range to analyse (e.g. `('2015-01-01', '2019-09-01')`).
Note that Sentinel-2 data is not available in Australia prior to 2015.
For reasonable results, the range should span at least two years to prevent detecting seasonal changes.
* `time_baseline`: The date at which to split the total sample into two non-overlapping samples (e.g. `'2017-12-01'`).
For reasonable results, pick a date that is about halfway between the start and end dates specified in `time`.

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.
The example covers the Kowen Forest, a commercial pine plantation in the Australian Capital Territory.

**To run the notebook for a different area**, make sure Sentinel-2 data is available for the chosen area using the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule).
Use the drop-down menu to check both Sentinel-2A (`s2a_ard_granule`) and Sentinel-2B (`s2b_ard_granule`).

In [3]:
# Define the area of interest
latitude = (-35.271, -35.331)
longitude = (149.256, 149.356)

# Set the range of dates for the complete sample
time = ('2015-01-01', '2019-09-01')

# Set the date to separate the data into two samples for comparison
time_baseline = '2017-12-01'

## View the selected location
The next cell will display the selected area on an interactive map.
The red border represents the area of interest of the study.
Zoom in and out to get a better understanding of the area of interest.
Clicking anywhere on the map will reveal the latitude and longitude coordinates of the clicked point.

In [4]:
display_map(x=longitude, y=latitude)